In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 128
num_epochs = 50
exract_size = 2048
class_size = 61



# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
encoder = Encoder()
encoder = encoder.to(device)

data_loader = get_encoder_loader(transform_train, encoder, mode='test', batch_size=batch_size)

NameError: name 'encoder' is not defined

### init model

In [6]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)



classify = FinalClassify(exract_size, class_size)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(classify.parameters())

AttributeError: cannot assign module before Module.__init__() call

### time to train model

In [22]:
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        images, targets = next(iter(data_loader))
        
        images = images.to(device)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(images)
        
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch, num_epochs, i_step, total_step, loss.item())
        
        print('\r' + stats, end='')
        sys.stdout.flush()

    torch.save(classify_model.fc.state_dict(), os.path.join('./models', 'class_model_%d.pkl' % epoch))

Epoch [1/1], Step [38/38], Loss: 1.6045

### How good is the model?

In [23]:
df_refer = data_loader.dataset.refer

In [29]:
img, _ = data_loader.dataset[200]
img = img.unsqueeze(0).to(device)
classify_model(img).argmax().item()

58

In [30]:
df_refer.disease_class[200]

58

In [31]:
predict = []
for img, _ in data_loader.dataset:
    img = img.unsqueeze(0).to(device)
    p = classify_model(img).argmax().item()
    predict.append(p)

In [33]:
df_refer["predict"] = predict

In [35]:
df_refer['correct'] = df_refer.predict == df_refer.disease_class

In [38]:
df_refer.query('correct == True')

,disease_class,image_id,predict,correct
52,58,32f121cba601f5267a4912eac6f52388.jpg,58,True
54,58,d0299a1340a1392c0db0f0fed44c5c41.jpg,58,True
56,58,f9fe9399-5c63-4a75-b9c4-83e4e79e25fb___YLCV_GC...,58,True
57,58,37b3b5ff1c718b6407e4216da011faa5.jpg,58,True
58,58,a18743d1229537e12bfb7b9a6d88d3d3 - 副本.jpg,58,True
59,58,44e554252d392775f2fa21a30936e493.jpg,58,True
60,58,1773aedb-6486-4649-b912-4c0a5bd96a01___UF.GRC_...,58,True
61,58,c8c6c0bbac437c25fc0580deea72b19e.jpg,58,True
62,58,538b6523-416b-401b-b739-5b0f88f65b41___YLCV_NR...,58,True
63,58,9b5220a4bf8b64cd19d801c9933cd3e6.jpg,58,True
